## Test Neuroevolución

In [1]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
import pandas as pd
from sklearn.preprocessing import StandardScaler  

In [2]:
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import random
import pickle
from copy import deepcopy
from sklearn import preprocessing

Importación del scaler
http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

### Cargo los datos y retiro el Timestamp y el score

In [4]:
df = pd.read_csv("gameStates.csv", sep=',', header=None)
df.columns = ['timeStamp','Px', 'Py', 'heat', 'Exp1','Eyp1','Exp2','Eyp2', 'Ex1', 'Ey1', 'Eh1', 'Ex2', 'Ey2', 'Eh2', 'Ex3', 'Ey3', 'Eh3', 'Ex4', 'Ey4','Eh4','Ex5', 'Ey5','Eh5','Ex6', 'Ey6','Eh6','ray1','ray2','ray3','ray4','ray5','ray6','ray7','ray8','ray9','ray10','ray11','ray12','ray13','ray14','ray15','ray16','ray17','ray18','ray19','ray20','ray21','ray22','ray23','ray24','ray25','ray26','ray27','score',"VKey","HKey","Shooting"]
df.drop(['timeStamp','score', 'Ex1', 'Ey1', 'Eh1', 'Ex2', 'Ey2', 'Eh2', 'Ex3', 'Ey3', 'Eh3', 'Ex4', 'Ey4','Eh4','Ex5', 'Ey5','Eh5','Ex6', 'Ey6','Eh6'],axis=1,inplace=True)
df = df[1:6001]


#### Transforma el porblema multioputput en multilabel

In [5]:
def multioutput2multilabel(row):
    values = pd.Series([(row["VKey"] == "UpArrow"),(row["VKey"] == "DownArrow"),(row["HKey"] == "LeftArrow"),(row["HKey"] == "RightArrow"),row["Shooting"]])
    return pd.concat([row[:len(row)-3], values.astype(float)])


In [6]:
def concatenar_estados(dataframe):
        dfAnterior = dataframe.shift(1)
        dfAnterior.rename(columns=lambda x: str(x)+"-1", inplace=True)
        dfAnterior2 = dataframe.shift(2)
        dfAnterior2.rename(columns=lambda x: str(x)+"-2", inplace=True)
        dfAnterior3 = dataframe.shift(3)
        dfAnterior3.rename(columns=lambda x: str(x)+"-3", inplace=True)
        df = pd.concat([dfAnterior3,dfAnterior2,dfAnterior,dataframe],axis=1)
        df = df.fillna(999)
        return df


df = df.apply(multioutput2multilabel,axis=1)
len(df.ix[1])

39

In [7]:
df = concatenar_estados(df)
len(df.ix[1])

156

In [9]:
#Defino los elementos train_data y target_data
train = df.drop([0, 1, 2,3,4],axis=1).values
labels = df[[0,1,2,3,4]].values

In [11]:
#define el perceptrón y lo entrena
#clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
#                    hidden_layer_sizes=(15,), random_state=1)
clf = MLPClassifier(solver='lbfgs', alpha=10, learning_rate= 'constant',
                    random_state = 0, learning_rate_init = 1, 
                    hidden_layer_sizes=(5,), batch_size = 200,) 
           

clf.fit(train, labels)
model = deepcopy(clf)
pickle.dump(clf, open(".\\AI_Data\\clf.sav", 'wb'))
pickle.dump(model, open(".\\AI_Data\\model.sav", 'wb'))
shapes = [coef.shape for coef in model.coefs_] # solo voy a ajustar los pesos
sizes =[coef.size for coef in model.coefs_]

In [12]:
#Neuroevolución

def gen2Coefs(gen,sizes,shapes):
    coefs = []
    splits = np.split(gen, [sizes[0]])
    for i in range(len(splits)):
        coefs.append(splits[i].reshape(shapes[i]))
    return coefs

def coefs2gen(coefs,sizes,shapes):
    return np.concatenate((coefs[0].flatten(),coefs[1].flatten()))


print(sizes)

[755, 25]


### Pipeline

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

clf = MLPClassifier(solver='lbfgs', alpha=10, learning_rate= 'constant',
                    random_state = 0, learning_rate_init = 1, 
                    hidden_layer_sizes=(5,), batch_size = 200,) 

estimators = [('stantandarize', StandardScaler()), ('clf', clf)]

pipe = Pipeline(estimators)

# entreno el pipe como si fuera un clasificador, 
# también podría hacer validación cruzada y todo como si fuese un clasificador normal
pipe.fit(train, labels)
pickle.dump(model, open(".\\AI_Data\\pipe.sav", 'wb'))


#from sklearn.model_selection import cross_val_score
#print(cross_val_score(pipe, train, labels, cv=5).mean())

In [14]:
def MLPFitness(individual):
    coefs = gen2Coefs(np.array(individual),sizes,shapes)
    pipe.named_steps['clf'].coefs_ = coefs 
    pickle.dump(model, open(".\\AI_Data\\model.sav", 'wb'))
    os.system("AI.exe")
    scores = pd.read_csv("myData.csv", sep=',', header=None)
    os.remove("myData.csv")
    f = np.mean(scores.values)
    return (f,)



#MLPFitness(coefs2gen(clf.coefs_,sizes,shapes))

In [15]:
def initPopulation(pop, ind_random, ind_guess,n_guess=1,n=10):
    
    pop = []
    n_random = n - n_guess
    for i in range(n_random):
        pop.append(ind_random())
        
    for i in range(n_guess):
        pop.append(ind_guess())
    
    return pop

In [ ]:
N = 1

pesosIniciales =  pickle.load(open("best1.sav", 'rb'))
pesosIniciales = coefs2gen(pipe.named_steps['clf'].coefs_,sizes,shapes).copy()
shapes = [coef.shape for coef in model.coefs_] # solo voy a ajustar los pesos
sizes =[coef.size for coef in model.coefs_]


creator.create("FitnessMax", base.Fitness, weights=(1.0,)) # Maximiza
creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=sum(sizes))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("individual_guess", lambda :creator.Individual(pesosIniciales))
toolbox.register("population_mix",initPopulation,list,toolbox.individual, toolbox.individual_guess)       


toolbox.register("evaluate", MLPFitness)
toolbox.register("mate", tools.cxOnePoint)

toolbox.register("mutate", tools.mutGaussian, mu=0.5, sigma=0.5, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=10)

random.seed(64)
  
population = toolbox.population_mix(n=10, n_guess=N)  
    
hof = tools.HallOfFame(1, similar=np.array_equal)
    
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)
   
population, logbook = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=1, stats=stats, halloffame=hof)
print(hof) # hall of the fame contiene el mejor individuo vivo en cada generacion
mejorInd = hof[0] # esta ordenado de manera que el primer elemento es el mejor de siempre

pickle.dump(mejorInd, open(".\\best1.sav", 'wb'))
MLPFitness(mejorInd)

gen	nevals	avg    	std    	min     	max    
0  	10    	20.7132	20.5401	-3.01483	60.7161


0  	5     	-180.767	80.8397	-287.219	-95.0069
1  	2     	-133.332	52.4052	-227.667	-95.0069
2  	3     	-158.725	53.4109	-213.969	-95.0069

In [21]:
coefs = gen2Coefs(np.array(mejorInd),sizes,shapes)
pipe.named_steps['clf'].coefs_ = coefs 
pickle.dump(model, open(".\\best1.sav", 'wb'))

In [20]:
print(hof)

[Individual([ 0.20447112,  0.00775108,  0.15211689,  0.55796475,  0.72296187,
        0.92459415,  0.34060226,  0.52013013,  0.09331633,  0.95036604,
        0.13711507,  0.3805449 ,  0.66629941,  0.92033171,  0.69372702,
        0.39206606,  0.23812315,  0.82489498,  0.17155091,  0.42651996,
        0.17142916,  0.89763566,  0.08524603,  0.01455098,  0.91275495,
        0.34434219,  0.63436724,  0.95702355,  0.25309347,  0.44086234,
        0.18544689,  0.62065847,  0.84601957,  0.95496715,  0.96444054,
        0.48112754,  0.56909003,  0.25591451,  0.48361452,  0.88819862,
        0.54515804,  0.18596798,  0.73534453,  0.59228706,  0.39585865,
        0.9756744 ,  0.13268453,  0.32261975,  0.98446772,  0.52220431,
        0.87739295,  0.18448076,  0.27228447,  0.01775943,  0.37538751,
        0.77705708,  0.29558482,  0.69791544,  0.91197936,  0.56826454,
        0.56322955,  0.69445096,  0.08979084,  0.96243306,  0.4141705 ,
        0.96347437,  0.43653574,  0.86695578,  0.05443809,

In [19]:
MLPFitness(mejorInd)

(29.493012000000004,)

### Calibrado de parámetros

In [28]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

from scipy.stats import randint as sp_randint

# Constructor del random forest
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(15,), random_state=1)



# especifica los parámetros y su distribución
## tamaño capa oculta
## Learning rate intit
param_dist = {"hidden_layer_sizes":[(2,),(5,),(10,),(50,),(100,)],
              "solver":['lbfgs'],
              "batch_size":[100,200,500],
              "learning_rate_init":[0.001,0.0001,0.0005,0.1,1],
              "learning_rate":["constant", "invscaling", "adaptive"],
              "alpha":[0.001,0.0001,0.0005,0.1,1,5,10],
              "random_state":[0,1,10]}

# ejecuta búsqueda aleatoria
num_iteraciones = 200 ## aumentar numero
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=num_iteraciones)

#clf.get_params().keys()
random_search.fit(train, labels)
send_message("Done!", 4191538)
send_message(random_search.best_params_,4191538)

In [29]:
print("Los mejores parámetros encontrados son:")
print()
print(random_search.best_params_)

Los mejores parámetros encontrados son:

{'solver': 'lbfgs', 'random_state': 0, 'learning_rate_init': 1, 'learning_rate': 'constant', 'hidden_layer_sizes': (5,), 'batch_size': 200, 'alpha': 10}


### Telegram bot

In [11]:
import json 
import requests

TOKEN = "371099199:AAFy_hyhXjc4Ew8pGS1engI6RdJ3YDT0VSU"
URL = "https://api.telegram.org/bot{}/".format(TOKEN)

In [14]:
def get_url(url):
    response = requests.get(url)
    content = response.content.decode("utf8")
    return content

def get_json_from_url(url):
    content = get_url(url)
    js = json.loads(content)
    return js

def get_updates():
    url = URL + "getUpdates"
    js = get_json_from_url(url)
    return js

def get_last_chat_id_and_text(updates):
    num_updates = len(updates["result"])
    last_update = num_updates - 1
    text = updates["result"][last_update]["message"]["text"]
    chat_id = updates["result"][last_update]["message"]["chat"]["id"]
    return (text, chat_id)
    print (get_last_chat_id_and_text(updates))

def send_message(text, chat_id):
    url = URL + "sendMessage?text={}&chat_id={}".format(text, chat_id)
    get_url(url)

In [ ]:
import time
def main():
    last_textchat = (None, None)
    while True:
        text, chat = get_last_chat_id_and_text(get_updates())
        if (text, chat) != last_textchat:
            send_message(text, chat)
            last_textchat = (text, chat)
        time.sleep(0.5)
main()

In [15]:
text, chat = get_last_chat_id_and_text(get_updates())
#send_message(text, chat)
print(chat)
send_message("Hola!", 4191538)

4191538
